# Nosql

## redis 主从复制（读写分离）
- [具体操作](https://github.com/jackfrued/Python-100-Days/blob/master/Day66-75/03.%E5%AD%98%E5%82%A8%E6%95%B0%E6%8D%AE.md)
- 在redis的路径下找到sentinel.conf
- 修改配置文件
    - 第69行设置主人的ip 和端口 以及选举是晋升主人需要的票数
    - 第71行设置主人的密码，以便哨兵和主人保持连接
    - 第98行设置主人崩溃多长时间后重选主人
- 启动哨兵
- redis sentinel.conf --sentinel
- 在实际工作，会有一个负载均衡器，一个负载均衡器下面有多个节点，而一组主从复制的redis,只是其中的一个节点，我们在程序中连接redis是连接负载均衡器的地址。这样我们就不用担心，更换master时更换地址
- 负载均衡
- 硬件：F5/A10
- 软件：
    - Nginx + 负载均衡算法
    - LVS
    - 避免单点故障
        - 负载均衡+双活
        - Keepalived:就相当于Redis的Sentinel

In [ ]:
# 用redis实现数据持久化
from hashlib import sha1
from urllib.parse import urljoin

import pickle
import re
import requests
import zlib

from bs4 import BeautifulSoup
from redis import Redis


def main():
    # 指定种子页面
    base_url = 'https://www.zhihu.com/'
    seed_url = urljoin(base_url, 'explore')
    # 创建Redis客户端
    client = Redis(host='1.2.3.4', port=6379, password='1qaz2wsx')
    # 设置用户代理
    headers = {'user-agent': 'Baiduspider'}
    # 通过requests模块发送GET请求并指定用户代理
    resp = requests.get(seed_url, headers=headers)
    # 创建BeautifulSoup对象并指定使用lxml作为解析器
    soup = BeautifulSoup(resp.text, 'lxml')
    href_regex = re.compile(r'^/question')
    # 将URL处理成SHA1摘要(长度固定更简短)
    hasher_proto = sha1()
    # 查找所有href属性以/question打头的a标签
    for a_tag in soup.find_all('a', {'href': href_regex}):
        # 获取a标签的href属性值并组装完整的URL
        href = a_tag.attrs['href']
        full_url = urljoin(base_url, href)
        # 传入URL生成SHA1摘要
        # 因为如果直接用href_proto，每次更新的内容前面都会有前面的内容
        # 通过复制对象来使用对象 原型模式，比用构造器要快的多
        hasher = hasher_proto.copy()
        hasher.update(full_url.encode('utf-8'))
        field_key = hasher.hexdigest()
        # 如果Redis的键'zhihu'对应的hash数据类型中没有URL的摘要就访问页面并缓存
        if not client.hexists('zhihu', field_key):
            html_page = requests.get(full_url, headers=headers).text
            # 对页面进行序列化和压缩操作
            zipped_page = zlib.compress(pickle.dumps(html_page))
            # 使用hash数据类型保存URL摘要及其对应的页面代码
            client.hset('zhihu', field_key, zipped_page)
    # 显示总共缓存了多少个页面
    print('Total %d question pages found.' % client.hlen('zhihu'))
    # 设置数据的存活时间，可以用ttl在redis中查看存活时间
    client.expire('zhihu', 500)


if __name__ == '__main__':
    main()

## 推送服务
- PC端 - websocket / SockJS / STOMP
- 移动端 - 第三方平台（极光、百度、小米、友盟）


## Mongodb

In [ ]:
"""
# mongodb基本操作
- use 数据库名 
- db.student.find()
- db.student.insert({})
- db.student.find({'age':{'gt':25, 'lt':30}}) 找到年龄大于25小于30的

前面是筛选条件，后面是要添加或修改的内容，如果有这个键，就修改，没有就添加
- db.student.update({'_id': 12},{'$set':{'tel': 1233}}) 


"""

- [mongodb教程](https://www.yiibai.com/mongodb/)

In [ ]:
# 用python操作mongodb
import pymongo


def main():
    # client = pymongo.MongoClient('mongo://47.98.172.171:27017')
    client = pymongo.MongoClient(host='47.98.172.171', port=27017)
    db = client.zhihu
    print(client)
    # 创建一个叫webpages的集合
    pages_cache = db.webpages
    # 插入一条数据，并返回这条数据的对象
    page_id = pages_cache.insert_one({'url': 'http:www.baidu.com'})
    # 返回这条数据的id
    print(page_id.inserted_id)
    # 删除{'url': 'http:www.baidu.com'}这条数据
    # print(pages_cache.remove({'url': 'http:www.baidu.com'}))
    # 一次性插入多条命令
    pages_cache.insert_many([
        {'_id': 1, 'url': 'http://www.baidu.com ',
         'content': 'shit'},
        {'_id': 2, 'url': 'http://www.qq.com ',
         'content': 'another shit'},
        {'_id': 3, 'url': 'http://www.qfedu.com ',
        'content': 'biggest shit'}
    ])
    # 更新数据，$set 表示在原有数据基础上进行添加或更新
    # upsert=True 表示如果没有这条数据就插入，如果有就进行更新
    print(pages_cache.update({'_id': 5}, {'$set': {'content': 'hello, world!'}}, upsert=True))
    # 查找集合中共有多少文档
    print(pages_cache.find().count())
    # 将找到的文档按id排序
    for doc in pages_cache.find().sort('_id'):
        print(doc)

    # 支持插入文档套文档的文档
    pages_cache.insert_one({
        'url': 'http://www.baidu.com',
               'content': 'bull shit!',
        'owner': {
            'name': 'Lee Yanhong',
            'age': 50,
            'idcard': '110220196804091203'
        }
    })


if __name__ == '__main__':
    main()
